In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('output.csv')

In [31]:
df_sort = df.sort_values('Freq',ascending=True)

In [32]:
names = [name.split('（')[0] for name in df_sort['Name'].tolist()]
links = df_sort['Link'].tolist()

In [8]:
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

driver.get(links[0])
driver.find_element_by_link_text('はい').click()
    

In [ ]:
# 正規表現を使う
import re

def isint2(s):  # 正規表現を使って判定を行う
    p = '[-+]?\d+'
    return True if re.fullmatch(p, s) else False

def isfloat2(s):  # 正規表現を使って判定を行う
    p = '[-+]?(\d+\.?\d*|\.\d+)([eE][-+]?\d+)?'
    return True if re.fullmatch(p, s) else False

In [29]:
def collect_title(name,url):
    data = {'Title': [],
            'Rate':[],
            'Image':[],
            'Link':[]}
    df = pd.DataFrame(data)
    
    driver.get(url)
    ninzu = driver.find_element_by_class_name('list-boxcaptside.list-boxpagenation').text
    num = -(-int(ninzu.split("\u3000")[0][0:-5].replace(',',''))//120)
    for i in range(num):
        driver.get(url  + '/page=' + str(i+1))
        elements = driver.find_element_by_class_name('d-item').find_elements_by_tag_name('img')

        for element in elements:
            src = element.get_attribute('src')
            alt = element.get_attribute('alt')
            rate = element.find_element_by_xpath('../../../..').find_element_by_class_name('rate').text
            ref = element.find_element_by_xpath('../..') 
            link= ref.get_attribute('href')
            se = pd.Series([alt,rate,src,link],['Title','Rate','Image','Link'])
            df = df.append(se, ignore_index=True)

    df.to_csv('../Picture/data/'+name+'.csv',index=False)   

In [ ]:
for i in range(0,len(links)):
    collect_title(names[i],links[i])